In [1]:
library(lme4)
library(data.table)

Loading required package: Matrix



In [2]:
n = 1000
n_iter = 2

effect_size_metric = c('ALT','COHEND', 'MEAN')
normalization_type = c('RUV')

In [3]:
get_glm_results<-function(d){
    results = coef(summary(glm(Discordance ~  Effect.Size * Sample.Quality, data = d)))
    col1 = rep(q, times = dim(results)[1])
    col2 = rep(p_, times = dim(results)[1])
    col3 = rownames(results)
    results = cbind(col1,col2,col3,results)
    colnames(results) = colnames(stats)
    results = data.frame(results)
    return(results)
}

In [4]:
# initialize dfs
stats_ = data.frame(matrix(ncol = 9, nrow = 0))
stats_.high = data.frame(matrix(ncol = 9, nrow = 0))
stats_.low = data.frame(matrix(ncol = 9, nrow = 0))
exp.res_ = data.frame(matrix(ncol = 13, nrow = 0))



for (nt in normalization_type){
    for (esm in effect_size_metric){
        
        # open df
        string = paste0('../data/', nt, 'concordance_df_forGLM_sizeandquality')
        string = paste0(string, '_effectsizemetric_', esm, '_n_', n,'_n_iter_', n_iter,'.csv')
        D = read.csv(string, row.names = 1)
        
        # initialize GLM dfs
        stats <- data.frame(matrix(ncol = 7))
        colnames(stats) = c('Sample Quality Metric', 'Subset Size', 'GLM Parameter', 'Estimate', 'Standard Error', 
                           't value', 'Pr(>|t|)')

        stats.low<-copy(stats)

        stats.high<-copy(stats)
        
        # collect GLM results
        count<-0
        for (q in c('Avg Freeze Thaw', 'Avg RIN')){
            for (p_ in unique(D$Subset.Size)){
                d = D[which((D['Metric'] == q) & (D['Subset.Size'] == p_)),]

                # interactive GLM
                results = get_glm_results(d)
                # concat
                if (count == 0 ){stats = results}
                else{stats = rbind(stats, data.frame(results))}

                # only low quality samples
                results.low = get_glm_results(d[which(d['Sample.Quality'] == 0),])
                if (count == 0 ){stats.low = results.low}
                else{stats.low = rbind(stats.low, data.frame(results.low))}

                # only high quality samples
                results.high = get_glm_results(d[which(d['Sample.Quality'] == 1),])
                if (count == 0 ){stats.high = results.high}
                else{stats.high = rbind(stats.high, data.frame(results.high))}

                count<-count + 1}}

        {rownames(stats) = seq(from = 1, to = dim(stats)[1], by = 1)}
        {rownames(stats.low) = seq(from = 1, to = dim(stats.low)[1], by = 1)}
        {rownames(stats.high) = seq(from = 1, to = dim(stats.high)[1], by = 1)}

        n.iter = count
        
        #summarize results
        n.descriptive.cols = 2 # first two columns
        col.names <- c('Sample Quality Metric', 'Subset Size',
                                      'GLM1: E*Q < E?', 'GLM1: Significance?', 'GLM1: Expected Result?',
                                       'GLM2: E_H < E_L', 'GLM2: Significance?','GLM2: Expected Result?',
                                      'GLM Inequality Agreement', 'GLM Inequality Agreement and Significance', 
                      'GLM All 1s')


        exp.res = data.frame(matrix(0, ncol = length(col.names), nrow = n.iter))
        colnames(exp.res) = col.names



        count <- 1
        for (q in c('Avg Freeze Thaw', 'Avg RIN')){
            for (p_ in unique(D$Subset.Size)){
                d = stats[(which(stats['Subset.Size'] == p_ & stats['Sample.Quality.Metric'] == q)),]
                E.Q = as.double(as.vector(d[4, 'Estimate'])) #interaction coefficient
                E.Q.p = as.double(as.vector(d[4, 7])) 
                E = as.double(as.vector(d[2, 'Estimate'])) # effect size coefficient
                E.p = as.double(as.vector(d[2, 7]))

                d.high = stats.high[(which(stats.high['Subset.Size'] == p_ & stats.high['Sample.Quality.Metric'] == q)),]
                E.H = as.double(as.vector(d.high[2, 'Estimate'])) # effect size coefficient
                E.H.p = as.double(as.vector(d.high[2, 7]))

                d.low = stats.low[(which(stats.low['Subset.Size'] == p_ & stats.low['Sample.Quality.Metric'] == q)),]
                E.L = as.double(as.vector(d.low[2, 'Estimate'])) # effect size coefficient
                E.L.p = as.double(as.vector(d.low[2, 7]))

                exp.res[count, 'Sample Quality Metric'] = q
                exp.res[count, 'Subset Size'] = p_


                if(E.Q < E){exp.res[count, 'GLM1: E*Q < E?'] = 1}
                if ((E.Q.p < 0.05) & (E.p < 0.05)){exp.res[count, 'GLM1: Significance?'] = 1}
                if (sum(as.numeric(exp.res[count, c('GLM1: E*Q < E?', 'GLM1: Significance?')])) == 2){
                    exp.res[count, 'GLM1: Expected Result?'] = 1
                }

                if (E.H < E.L){exp.res[count, 'GLM2: E_H < E_L'] = 1}
                if ((E.H.p < 0.05) & (E.L.p < 0.05)){exp.res[count, 'GLM2: Significance?'] = 1}
                if (sum(as.numeric(exp.res[count, c('GLM2: E_H < E_L', 'GLM2: Significance?')])) == 2){
                    exp.res[count, 'GLM2: Expected Result?'] = 1
                }

                if (exp.res[count, 'GLM1: E*Q < E?'] == exp.res[count, 'GLM2: E_H < E_L']){
                    exp.res[count, 'GLM Inequality Agreement'] = 1
                }

                if (sum(as.numeric(exp.res[count, c('GLM Inequality Agreement', 
                                                    'GLM2: Significance?','GLM2: Significance?')])) == 3){
                    exp.res[count, 'GLM Inequality Agreement and Significance'] = 1
                }

                cols.of.interest = (n.descriptive.cols + 1):(length(col.names) - 1)
                if (sum(as.numeric(exp.res[count,cols.of.interest])) == length(cols.of.interest)){
                    exp.res[count, 'GLM All 1s'] = 1
                }

                count<-count + 1
            }
        }
        stats = cbind(esm, nt, stats)
        stats.low = cbind(esm, nt, stats.low)
        stats.high = cbind(esm, nt, stats.high)
        exp.res = cbind(esm, nt, exp.res)
        
        stats_ = rbind(stats_,stats)
        stats_.low = rbind(stats_.low, stats.low)
        stats_.high = rbind(stats_.high, stats.high)
        exp.res_ = rbind(exp.res_, exp.res)
        
        
        # save files
        labels = list('_GLM_all_', '_GLM_high_', '_GLM_low__', '_GLM_results_summary_')
        results_ = list(stats_, stats_.low, stats_.high, exp.res)

        }
    }


In [5]:
string = paste0('../data/supplementary_discordance_GLM_2')
string = paste0(string, '_n_', n,'_n_iter_', n_iter,'.csv')
write.csv(results_[[1]], string) # BH correction later

# Subset Size Only

In [31]:
get_glm_results<-function(d){
    results = coef(summary(glm(Discordance ~  Effect.Size, data = d)))
    # format results
    col1 = rep(q, times = dim(results)[1])
    col2 = rep(p_, times = dim(results)[1])
    col3 = rownames(results)
    results = cbind(col1,col2,col3,results)
    colnames(results) = colnames(stats)
    results = data.frame(results)
    return(results)
}

In [35]:
# initialize dfs
stats_ = data.frame(matrix(ncol = 9, nrow = 0))


for (nt in normalization_type){
    for (esm in effect_size_metric){
        
        # open df
        string = paste0('/data/hratch/RNAseqQC/data/', nt, 'concordance_df_forGLM_sizeonly')
        string = paste0(string, '_effectsizemetric_', esm, '_n_', n,'_n_iter_', n_iter,'.csv')
        D = read.csv(string, row.names = 1)
        
        # initialize GLM dfs
        stats <- data.frame(matrix(ncol = 7))
        colnames(stats) = c('Sample Quality Metric', 'Subset Size', 'GLM Parameter', 'Estimate', 'Standard Error', 
                           't value', 'Pr(>|t|)')

        stats.low<-copy(stats)

        stats.high<-copy(stats)
        
        # collect GLM results
        count<-0
        for (p_ in unique(D$Subset.Size)){
            d = D[which(D['Subset.Size'] == p_),]

            # interactive GLM
            results = get_glm_results(d)
            # concat
            if (count == 0 ){stats = results}
            else{stats = rbind(stats, data.frame(results))}

            count<-count + 1}

        {rownames(stats) = seq(from = 1, to = dim(stats)[1], by = 1)}

        stats = cbind(esm, nt, stats)
        stats_ = rbind(stats_,stats)
        

        
        
    }  
}

  

In [45]:
stats_ = stats_[,which(colnames(stats_) != "Sample.Quality.Metric")]
string = paste0('../data/supplementary_discordance_GLM_1')
string = paste0(string, '_n_', n,'_n_iter_', n_iter,'.csv')
write.csv(stats_, string)